In [136]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [137]:
verbose=True

In [138]:
df = pd.read_csv(r'csv/enrollments.csv')

/var/folders/84/w6979yl547ddlndc7496yzsw0000gn/T/ipykernel_32111/3555196855.py:1: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'csv/enrollments.csv')


In [139]:
# Convert year float to dt object
df['year'] = pd.to_datetime(df['year'], format='%Y')
df['year'] = df['year'].apply(lambda x: pd.to_datetime(x))

# df['foundedyr'] = pd.to_datetime(df['foundedyr'], format='%Y',errors = 'coerce')
# df['foundedyr'] = df['foundedyr'].apply(lambda x: pd.to_datetime(x))


In [140]:
df.head()

,country,countrycode,region,incomegroup,iau_id,iau_id1,eng_name,orig_name,foundedyr,yrclosed,...,divisions,total_fields,unique_fields,specialized,merger,noiau,year,students5_interpolated,students5_extrapolated,students5_estimated
0,afghanistan,AFG,South Asia,Low income,IAU-000810,IAU-000810-1,Alberoni University,Alberoni University,1998.0,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,2000-01-01,NaN,1546.0,1546.0
1,afghanistan,AFG,South Asia,Low income,IAU-000810,IAU-000810-1,Alberoni University,Alberoni University,1998.0,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,2005-01-01,NaN,2106.0,2106.0
2,afghanistan,AFG,South Asia,Low income,IAU-000810,IAU-000810-1,Alberoni University,Alberoni University,1998.0,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,2010-01-01,2317.0,2317.0,2317.0
3,afghanistan,AFG,South Asia,Low income,IAU-000810,IAU-000810-1,Alberoni University,Alberoni University,1998.0,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,2015-01-01,1835.0,1835.0,1835.0
4,afghanistan,AFG,South Asia,Low income,IAU-000810,IAU-000810-1,Alberoni University,Alberoni University,1998.0,NaN,...,9.0,22.0,21.0,0.0,0.0,0.0,2020-01-01,NaN,1901.0,1901.0


In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161560 entries, 0 to 161559
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   country                 161560 non-null  object        
 1   countrycode             159698 non-null  object        
 2   region                  159698 non-null  object        
 3   incomegroup             159691 non-null  object        
 4   iau_id                  161553 non-null  object        
 5   iau_id1                 161553 non-null  object        
 6   eng_name                161553 non-null  object        
 7   orig_name               138474 non-null  object        
 8   foundedyr               161553 non-null  float64       
 9   yrclosed                460 non-null     float64       
 10  private01               161553 non-null  float64       
 11  coordinates             138352 non-null  object        
 12  latitude                138352

In [142]:
# Drop useless columns
df.drop(['iau_id', 'iau_id1','yrclosed','orig_name','students5_interpolated','students5_extrapolated','divisions','total_fields','merger','unique_fields','noiau','specialized'], axis=1, inplace=True)
cols_name_map = {'country': 'Country', 'eng_name': 'University', 'foundedyr': 'Founded in', 'b_granting': 'Bachelor', 'm_granting': 'Master', 'phd_granting': 'PhD', 'students5_estimated': 'Number of Student'}
# Rename
df = df.rename(columns=cols_name_map)


In [143]:
# Fixing nan
df = df[~df['latitude'].isna()]
df = df[~df['longitude'].isna()]
df = df[~df['Number of Student'].isna()]

In [144]:
# format and refactor values to be more user-friendly and nice-to-see in the map
df['Country'] = df['Country'].apply(lambda x: x.title())
df['University'] = df['University'].apply(lambda x: x.title())
df['University'] = df['University'].apply(lambda x: x.replace('\\N', ' - ').title())

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127649 entries, 0 to 161559
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Country            127649 non-null  object        
 1   countrycode        127102 non-null  object        
 2   region             127102 non-null  object        
 3   incomegroup        127102 non-null  object        
 4   University         127649 non-null  object        
 5   Founded in         127649 non-null  float64       
 6   private01          127649 non-null  float64       
 7   coordinates        127649 non-null  object        
 8   latitude           127649 non-null  object        
 9   longitude          127649 non-null  object        
 10  PhD                127649 non-null  float64       
 11  Master             20506 non-null   float64       
 12  Bachelor           20506 non-null   float64       
 13  year               127649 non-null  datetime

## CALCULATE TOTAL STUDENTS FROM EACH COUNTRY EVERY YEAR.
STUDENTS X COUNTRY

In [146]:
grouped = df.groupby(['Country', 'year'])['Number of Student'].sum().unstack(0).reset_index()
grouped = grouped.fillna(0)
grouped = grouped.rename(columns={'year':'time'})
grouped.to_csv('csv/processed/students_by_country_year.csv', index=False)

## FIND STUDENTS IN EACH INCOME GROUP FROM FOR EVERY COUNTRY 

In [152]:
df.incomegroup.value_counts()

Upper middle income    48007
High income            47556
Lower middle income    27116
Low income              4423
Name: incomegroup, dtype: int64

## Count enrolments from Each Country

In [86]:
df['Number of Student'] = df['Number of Student'].fillna(0).apply(int)

In [181]:
# Count enrolments from each country

country_counts = df.groupby(['Country','countrycode','region'])['Number of Student'].agg('sum').reset_index()
country_counts = country_counts.sort_values(by='Number of Student')
country_counts = country_counts.rename(columns={'Number of Student': 'students'})
country_counts.to_csv('csv/processed/country_enrolment_processed.csv',index=False)
country_counts

,Country,countrycode,region,students
148,Seychelles,SYC,Sub-Saharan Africa,237.0
141,Saint Kitts And Nevis,KNA,Latin America and Caribbean,239.0
170,Tonga,TON,East Asia and Pacific,324.0
144,Sao Tome And Principe,STP,Sub-Saharan Africa,371.0
38,Comoros,COM,Sub-Saharan Africa,646.0
...,...,...,...,...
139,Russian Federation,RUS,Europe and Central Asia,34933929.0
134,Philippines,PHL,East Asia and Pacific,38929878.0
179,United States,USA,North America,53203189.0
35,China,CHN,East Asia and Pacific,63595771.0


## Enrolments from each region

In [88]:
# Count enrolments from each country
region_counts = df.groupby(['region'])['Number of Student'].agg('sum').reset_index()
# country_counts = country_counts.sort_values(by='Number of Student')
region_counts = region_counts.rename(columns={'Number of Student': 'students'})
region_counts.to_csv('csv/processed/region_enrolment_processed.csv',index=False)
region_counts

,region,students
0,East Asia and Pacific,213809094
1,Europe and Central Asia,224219259
2,Latin America and Caribbean,134933052
3,Middle East and North Africa,58774877
4,North America,68942778
5,South Asia,112108700
6,Sub-Saharan Africa,30113437


## Enrolments from each country

In [89]:
# Count enrolments from each country

region_counts = df.groupby(['countrycode'])['Number of Student'].agg('sum').reset_index()
region_counts = region_counts.rename(columns={'Number of Student': 'students'})
region_counts.to_csv('csv/processed/country_student_count.csv',index=False)
region_counts

,countrycode,students
0,ABW,3870
1,AFG,555026
2,AGO,588468
3,ALB,558980
4,AND,7431
...,...,...
182,WSM,12020
183,YEM,599588
184,ZAF,4938872
185,ZMB,270518


## Count enrolment from Each Income Group

In [90]:
# Count enrolments from each income group
income_group_counts = df.groupby(['incomegroup'])['Number of Student'].agg('sum').reset_index()
# Normalize the values between 0 and 100 in %.
total_sum = income_group_counts['Number of Student'].sum()
income_group_counts['count_percentage'] = income_group_counts['Number of Student'].apply(lambda x: (x / total_sum) * 100)
# Write to csv
income_group_counts.to_csv('csv/processed/income_group_count.csv',index=False)
income_group_counts

,incomegroup,Number of Student,count_percentage
0,High income,279100784,33.111922
1,Low income,21003981,2.491868
2,Lower middle income,228999209,27.167978
3,Upper middle income,313797223,37.228233


## Total enrolments yearwise and from each incomegroup

In [230]:
# income_group_by_year = df.groupby(['year', 'incomegroup'])['Number of Student'].agg('sum').reset_index()
income_group_by_year = df.groupby(['year', 'incomegroup']).agg({'Number of Student': 'sum'}).unstack(level=1)
income_group_by_year.columns=income_group_by_year.columns.droplevel()
income_group_by_year = income_group_by_year.reset_index()
#income_group_by_year.to_csv('csv/processed/income_group_by_year.csv')

In [231]:
income_group_by_year

incomegroup,year,High income,Low income,Lower middle income,Upper middle income
0,1950-01-01,3865348.0,9749.0,675920.0,830549.0
1,1955-01-01,4521373.0,19926.0,908247.0,1168170.0
2,1960-01-01,5379428.0,82146.0,1659381.0,2244256.0
3,1965-01-01,6686537.0,147164.0,2671820.0,3220454.0
4,1970-01-01,8705792.0,258139.0,4152659.0,4657680.0
5,1975-01-01,11160007.0,397690.0,6564630.0,7044174.0
6,1980-01-01,13402261.0,576380.0,8141100.0,9598727.0
7,1985-01-01,15463393.0,697138.0,10326365.0,12564164.0
8,1990-01-01,18557702.0,877582.0,12724344.0,15410818.0
9,1995-01-01,22126059.0,1136080.0,15941731.0,19474968.0


## Count enrolments yearwise

In [93]:
# year wise enrolment
year_wise = df.groupby(['year'])['Number of Student'].agg('sum').reset_index()
year_wise.to_csv('csv/processed/yearwise_enrolment.csv',index=False)
year_wise

,year,Number of Student
0,1950-01-01,5386106
1,1955-01-01,6630699
2,1960-01-01,9384192
3,1965-01-01,12753551
4,1970-01-01,17822731
5,1975-01-01,25238733
6,1980-01-01,31805364
7,1985-01-01,39147983
8,1990-01-01,47695827
9,1995-01-01,58865676


## Yearwise Private / Public split

In [212]:
private_public_by_year = df.groupby(['year', 'private01'])['Number of Student'].agg('sum').unstack(1).reset_index()
private_public_by_year = private_public_by_year.rename(columns = {0.0:'public',1.0:'private'})
private_public_by_year.to_csv('csv/processed/private_by_year.csv',index=False)
private_public_by_year

private01,year,public,private
0,1950-01-01,4188509.0,1197597.0
1,1955-01-01,5222437.0,1408262.0
2,1960-01-01,7410754.0,1973438.0
3,1965-01-01,10141061.0,2612490.0
4,1970-01-01,14278199.0,3544532.0
5,1975-01-01,20535151.0,4703582.0
6,1980-01-01,25831690.0,5973674.0
7,1985-01-01,31447302.0,7700681.0
8,1990-01-01,37786503.0,9909324.0
9,1995-01-01,45867791.0,12997885.0


## Merge private-public and income group yearwise data.

In [235]:
yearwise_merged = pd.merge(income_group_by_year,private_public_by_year,on='year')
cols_name_map = {'High income':'high','Low income':'low_income','Lower middle income':'low_middle_income','Upper middle income':'upper_middle_income'}
yearwise_merged = yearwise_merged.rename(columns=cols_name_map)
#rename the columns 
yearwise_merged.

,year,high,low_income,low_middle_income,upper_middle_income,public,private
0,1950-01-01,3865348.0,9749.0,675920.0,830549.0,4188509.0,1197597.0
1,1955-01-01,4521373.0,19926.0,908247.0,1168170.0,5222437.0,1408262.0
2,1960-01-01,5379428.0,82146.0,1659381.0,2244256.0,7410754.0,1973438.0
3,1965-01-01,6686537.0,147164.0,2671820.0,3220454.0,10141061.0,2612490.0
4,1970-01-01,8705792.0,258139.0,4152659.0,4657680.0,14278199.0,3544532.0
5,1975-01-01,11160007.0,397690.0,6564630.0,7044174.0,20535151.0,4703582.0
6,1980-01-01,13402261.0,576380.0,8141100.0,9598727.0,25831690.0,5973674.0
7,1985-01-01,15463393.0,697138.0,10326365.0,12564164.0,31447302.0,7700681.0
8,1990-01-01,18557702.0,877582.0,12724344.0,15410818.0,37786503.0,9909324.0
9,1995-01-01,22126059.0,1136080.0,15941731.0,19474968.0,45867791.0,12997885.0


In [95]:
df.head()

,Country,countrycode,region,incomegroup,University,Founded in,private01,coordinates,latitude,longitude,PhD,Master,Bachelor,year,Number of Student
0,Afghanistan,AFG,South Asia,Low income,Alberoni University,1998.0,0.0,"35.1270053, 69.3193192",35.1270053,69.319319,0.0,NaN,NaN,2000-01-01,1546
1,Afghanistan,AFG,South Asia,Low income,Alberoni University,1998.0,0.0,"35.1270053, 69.3193192",35.1270053,69.319319,0.0,NaN,NaN,2005-01-01,2106
2,Afghanistan,AFG,South Asia,Low income,Alberoni University,1998.0,0.0,"35.1270053, 69.3193192",35.1270053,69.319319,0.0,NaN,NaN,2010-01-01,2317
3,Afghanistan,AFG,South Asia,Low income,Alberoni University,1998.0,0.0,"35.1270053, 69.3193192",35.1270053,69.319319,0.0,NaN,NaN,2015-01-01,1835
4,Afghanistan,AFG,South Asia,Low income,Alberoni University,1998.0,0.0,"35.1270053, 69.3193192",35.1270053,69.319319,0.0,0.0,1.0,2020-01-01,1901


## Count total universities in each country 

In [96]:
df.groupby(['Country','University'])['Country'].count()

Country      University                                                          
Afghanistan  Afghanistan National Agricultural Sciences And Technology University     2
             Air University                                                           3
             Alberoni University                                                      5
             American University Of Afghanistan                                       3
             Ariana University                                                        4
                                                                                     ..
Zimbabwe     University Of Zimbabwe                                                  14
             University Solution                                                     13
             Women'S University In Africa                                             4
             Zimbabwe Ezekiel Gut University                                          2
             Zimbabwe Open University 

## Count total universities in the world

In [97]:
df.University.nunique()

15426

In [98]:
#Count students studying in 2020
import datetime as dt
df['Date'] = pd.to_datetime(df['year'])

include = df[df['Date'].dt.year == 2020]


In [99]:
total_students_2020 = include.groupby(['year'])['Number of Student'].agg('sum').reset_index()

In [100]:
total_students_2020

,year,Number of Student
0,2020-01-01,163788588


In [133]:
df = df.fillna(0)

In [103]:
df.value_counts('region')

region
East Asia and Pacific           37827
Europe and Central Asia         36540
Latin America and Caribbean     19733
North America                   12470
South Asia                       7080
Sub-Saharan Africa               6967
Middle East and North Africa     6485
dtype: int64

# Create dataframe with the following information 
## Country Countrycode oldest-UniName year-founded bachelors masters phd low-income high-income etc 

In [104]:
# Find oldest university in each country
oldest_university = df.sort_values(by='Founded in').groupby('Country').first().reset_index()

# Print the result
oldest_university

rslt_df = oldest_university.loc[oldest_university['countrycode'] =='USA']
rslt_df

,Country,countrycode,region,incomegroup,University,Founded in,private01,coordinates,latitude,longitude,PhD,Master,Bachelor,year,Number of Student,Date
185,United States,USA,North America,High income,Harvard University,1636.0,1.0,"42.3770029, -71.1166601",42.3770029,-71.1166601,0.0,1.0,1.0,1965-01-01,18365,1965-01-01


### Count private and public uni in every country 

In [202]:
# Count the number of private and not private colleges for each country
count = df.groupby(['countrycode']).agg({'private01':['sum','size']}).reset_index()
count.columns = ['countrycode','Private','Not private']
count['Not private'] = count['Not private'] - count['Private']

# Print the result
count

,countrycode,Private,Not private
0,ABW,0.0,14.0
1,AFG,45.0,196.0
2,AGO,77.0,54.0
3,ALB,54.0,97.0
4,AND,3.0,7.0
...,...,...,...
182,WSM,0.0,17.0
183,YEM,26.0,47.0
184,ZAF,110.0,199.0
185,ZMB,95.0,41.0


## Count masters, bachelors and PhD for each country.


In [106]:
# Sum the values of the 'masters', 'bachelors', and 'phd' columns for each group of 'Country'
count_degrees = df.groupby('countrycode').agg({'Master': 'sum', 'Bachelor': 'sum', 'PhD': 'sum'}).reset_index()
cols_name_map = {'country': 'Country', 'Master': 'total_masters', 'Bachelor': 'total_bachelor', 'PhD': 'total_phd'}
count_degrees = count_degrees.rename(columns=cols_name_map)

# Calculate the total number of degrees for each row
count_degrees['total'] = count_degrees['total_masters'] + count_degrees['total_bachelor'] + count_degrees['total_phd']

# Calculate the percentage of each degree for each row
count_degrees['percent_masters'] = count_degrees['total_masters'] / count_degrees['total'] * 100
count_degrees['percent_bachelor'] = count_degrees['total_bachelor'] / count_degrees['total'] * 100
count_degrees['percent_phd'] = count_degrees['total_phd'] / count_degrees['total'] * 100

# Delete useless columns
count_degrees = count_degrees.drop(['total_masters','total_bachelor','total_phd'],axis=1)
count_degrees
# Show the resulting dataframe
# print(count_degrees)

,countrycode,total,percent_masters,percent_bachelor,percent_phd
0,ABW,8.0,50.000000,50.000000,0.000000
1,AFG,68.0,25.000000,66.176471,8.823529
2,AGO,43.0,0.000000,100.000000,0.000000
3,ALB,64.0,40.625000,42.187500,17.187500
4,AND,3.0,0.000000,0.000000,100.000000
...,...,...,...,...,...
182,WSM,10.0,40.000000,40.000000,20.000000
183,YEM,54.0,38.888889,61.111111,0.000000
184,ZAF,67.0,40.298507,29.850746,29.850746
185,ZMB,52.0,36.538462,42.307692,21.153846


## count universities in each country

In [205]:
# Group the data by country and count the number of universities
country_counts = df.groupby('countrycode').count()
# Get the number of universities for each country
university_counts = country_counts['University'].reset_index()
# rename column
cols_name_map = {'University':'total_universities'}
university_counts = university_counts.rename(columns=cols_name_map)
# university_counts.sort_values(by='total_universities', ascending=False)

### Merge both the datasets

In [206]:
# Merge the dataframes on the 'Country' column
merged_df = pd.merge(count, oldest_university, on='countrycode')
merged_df = pd.merge(merged_df,count_degrees,on='countrycode')
merged_df = pd.merge(merged_df,region_counts,on='countrycode')
merged_df = pd.merge(merged_df,university_counts,on='countrycode')
#convert to year
merged_df['year'] = pd.DatetimeIndex(merged_df['year']).year
merged_df['Date'] = pd.DatetimeIndex(merged_df['Date']).year


# rename the columns 
cols_name_map = {'Number of Student':'students_in_oldest','students':'total_students','Founded in':'founded_in','Not private':'Public'}
merged_df = merged_df.rename(columns=cols_name_map)
# merged_df['founded_in'] = pd.DatetimeIndex(merged_df['founded_in']).year

#remove useless columns 
merged_df = merged_df.drop(['Date','students_in_oldest','coordinates'],axis=1)

#write to csv
merged_df.info()
merged_df.to_csv('csv/processed/merged_dataset.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187 entries, 0 to 186
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   countrycode         187 non-null    object 
 1   Private             187 non-null    float64
 2   Public              187 non-null    float64
 3   Country             187 non-null    object 
 4   region              187 non-null    object 
 5   incomegroup         187 non-null    object 
 6   University          187 non-null    object 
 7   founded_in          187 non-null    float64
 8   private01           187 non-null    float64
 9   latitude            187 non-null    object 
 10  longitude           187 non-null    object 
 11  PhD                 187 non-null    float64
 12  Master              185 non-null    float64
 13  Bachelor            185 non-null    float64
 14  year                187 non-null    int64  
 15  total               187 non-null    float64
 16  percent_

In [207]:
rslt_df = merged_df.loc[merged_df['countrycode'] =='USA']
rslt_df

,countrycode,Private,Public,Country,region,incomegroup,University,founded_in,private01,latitude,...,PhD,Master,Bachelor,year,total,percent_masters,percent_bachelor,percent_phd,total_students,total_universities
178,USA,7541.0,3288.0,United States,North America,High income,Harvard University,1636.0,1.0,42.3770029,...,0.0,1.0,1.0,1965,4142.0,37.059392,41.91212,21.028489,53203189,10829


In [208]:
merged_df


,countrycode,Private,Public,Country,region,incomegroup,University,founded_in,private01,latitude,...,PhD,Master,Bachelor,year,total,percent_masters,percent_bachelor,percent_phd,total_students,total_universities
0,ABW,0.0,14.0,Aruba,Latin America and Caribbean,High income,University Of Aruba,1988.0,0.0,12.5234234,...,0.0,1.0,1.0,2015,8.0,50.000000,50.000000,0.000000,3870,14
1,AFG,45.0,196.0,Afghanistan,South Asia,Low income,Shaheed Rabbani Education University,1913.0,0.0,34.5323516,...,0.0,1.0,1.0,1970,68.0,25.000000,66.176471,8.823529,555026,241
2,AGO,77.0,54.0,Angola,Sub-Saharan Africa,Lower middle income,Kimpa Life University,1983.0,0.0,-7.617492,...,0.0,0.0,1.0,2000,43.0,0.000000,100.000000,0.000000,588468,131
3,ALB,54.0,97.0,Albania,Europe and Central Asia,Upper middle income,Sports University Of Tirana,1948.0,0.0,41.3307726,...,0.0,1.0,1.0,1955,64.0,40.625000,42.187500,17.187500,558980,151
4,AND,3.0,7.0,Andorra,Europe and Central Asia,High income,University Of Andorra,1988.0,0.0,42.4653237,...,0.0,0.0,0.0,2015,3.0,0.000000,0.000000,100.000000,7431,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,WSM,0.0,17.0,Samoa,East Asia and Pacific,Upper middle income,"University Of The South Pacific, School Of Agr...",1977.0,0.0,-18.1480462,...,1.0,1.0,1.0,2020,10.0,40.000000,40.000000,20.000000,12020,17
183,YEM,26.0,47.0,Yemen,Middle East and North Africa,Low income,Sana'A University.,1970.0,0.0,15.3485208,...,0.0,0.0,1.0,1985,54.0,38.888889,61.111111,0.000000,599588,73
184,ZAF,110.0,199.0,South Africa,Sub-Saharan Africa,Upper middle income,University Of Cape Town,1829.0,0.0,43.6005811,...,0.0,1.0,1.0,1960,67.0,40.298507,29.850746,29.850746,4938872,309
185,ZMB,95.0,41.0,Zambia,Sub-Saharan Africa,Lower middle income,National Institute Of Public Administration,1963.0,0.0,-15.4192514,...,0.0,1.0,1.0,1970,52.0,36.538462,42.307692,21.153846,270518,136


In [209]:
university_counts.sort_values(by='total_universities', ascending=False)

,countrycode,total_universities
178,USA,10829
32,CHN,8877
133,PHL,7572
86,JPN,7338
143,RUS,6795
...,...,...
156,STP,5
21,BLZ,4
38,COM,4
162,SYC,3
